In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
if not (os.path.exists('./data/userid-profile.tsv') and os.path.exists('./data/userid-timestamp-artid-artname-traid-traname.tsv')):
  raise ValueError('Los archivos userid-profile y userid-timestamp-artid-artname-traid-traname no fueron encontrados en el path')
else:
  print("Los archivos han sido cargados")

Los archivos han sido cargados


In [7]:
users=pd.read_csv('./data/userid-profile.tsv', sep = '\t', header=0 )

In [8]:
activity=pd.read_csv('./data/userid-timestamp-artid-artname-traid-traname.tsv', sep = '\t', names = [ 'userid', 'timestamp', 'musicbrainz-artist-id', 'artist-name', 'musicbrainz-track-id', 'track-name' ])

In [ ]:
activity.count()

### Luego contamos las veces en las que usuarios escucharon un artista para encontrar el percentil 75 y descartar

In [ ]:
user_artists_count = activity.groupby(['artist-name']).count()

In [ ]:
sorted_user_artists_count = user_artists_count['track-name'].reset_index(name='count')

In [ ]:
sorted_user_artists_count['count'].describe()

### Ahora eliminamos aquellos artistas que tienen menos de 18 actividades (percentil 75)

In [ ]:
noisyArtists = sorted_user_artists_count[sorted_user_artists_count['count'] <= 18]
noisyArtists = noisyArtists.loc[:,['artist-name']]

In [ ]:
noisy_df = activity.merge(noisyArtists, how="inner", on="artist-name")

In [ ]:
reduced_activity = pd.concat([activity, noisy_df])
reduced_activity = reduced_activity.drop_duplicates(keep=False)
reduced_activity.count()

### Calculamos la cantidad de veces que un usuario ha escuchado un artista por usuario

In [ ]:
user_artists_count = activity.groupby(['userid', 'artist-name']).count()

In [ ]:
sorted_user_artists_count = user_artists_count['track-name'].reset_index(name='count').sort_values(['userid', 'count'], ascending=False)
sorted_user_artists_count

### Por último sacamos un rating basado en la cantidad de veces que un artista fué escuchado de manera proporcional

In [ ]:
# obtenemos un dataframe con el maximo de reproducciones por cada usuario
user_max = pd.DataFrame(sorted_user_artists_count.groupby(['userid'], sort=False)['count'].max()).reset_index()
user_max.columns = ['userid', 'max']
user_max

In [ ]:
# hacemos merge con la tabla de usuarios y artistas
sorted_user_artists_count_max = sorted_user_artists_count.merge(user_max, how='left', left_on='userid', right_on='userid')

In [ ]:
sorted_user_artists_count_max

In [ ]:
# creamos una nueva columna en el dataframe con el ranking que va de 0 a 5, donde 5 es el artista mas oido y 0 el menos oido
sorted_user_artists_count_max['rating'] = np.around((sorted_user_artists_count_max['count']/sorted_user_artists_count_max['max']) * 5, decimals=1)


In [ ]:
sorted_user_artists_count_max

In [ ]:
sorted_user_artists_count_max.rating.hist()

In [ ]:
sorted_user_artists_count_max.rating.describe()

In [ ]:
np.percentile(sorted_user_artists_count_max.rating, 95)

In [ ]:
valuableRecords = sorted_user_artists_count_max[sorted_user_artists_count_max['rating'] > 0.7]
valuableRecords = valuableRecords.loc[:,['userid', 'artist-name', 'rating']]
valuableRecords

In [ ]:
valuableRecords.rating.hist()

In [ ]:
valuableRecords.to_csv('./data/preprocessed_user_item_rating.csv')